#MIP Merge Data Loss

This script documents the data lost in each step of the MIP Merge Jupyter Notebook

In [22]:
import pandas as pd
import numpy as np

Load the data into DataFrames

In [ ]:
df_ownership = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPOwn.csv", 
                           encoding="ISO-8859-1")
df_companies = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPMIP_panel_owned.csv", 
                           encoding="ISO-8859-1")
df_owners = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPMIP_panel_owner.csv", 
                        encoding="ISO-8859-1")

Create DataFrame to be populated by the data loss info

In [24]:
num_companyid = len(df_ownership["companyid"].unique())
print("Number of unique companyids:", f"{num_companyid:,}")
num_b_crefo = len(df_ownership["b_crefo"].unique())
print("Number of unique owners (b_crefo):", f"{num_b_crefo:,}")
num_ownerid = len(df_ownership["ownerid"].unique())
print("Number of unique owners (ownerid):", f"{num_ownerid:,}")
dict = {
    "Step" : "Load data",
    "companyid" : [num_companyid],
    "b_crefo" : [num_b_crefo],
    "ownerid" : [num_ownerid]
}
df_data_loss = pd.DataFrame(dict)

Number of unique companyids: 26,185
Number of unique owners (b_crefo): 227,833
Number of unique owners (ownerid): 10,092


While in the 2015 and 2021 the MIP survey questions regarding environmental innovation are in a scale from 0 to 2 (0=nein, 1=gering, 2=mittel), in 2009 the scale is from 0 to 3 (0=nein, 1=gering, 2=mittel, 3=hoch). Therefore, to make the answers consistent through the years, the 3's are going to turn into 2's, as "medium" and "high relevance" can be mapped to "important". In addition, i'll turn the text labels into numerical values.

In [25]:
environmental_variables = ["oekpz1", "oekpz2", "oekpz3", "oekpz4", "oekpz5", "oekpz6", "oekpz7", "oekpz8", "oekpz9", "oekpd1", "oekpd2", "oekpd3", "oekpd4"]
for i in range(len(environmental_variables)):
    df_companies[environmental_variables[i]] = np.where(df_companies[environmental_variables[i]] == "hoch", "mittel", df_companies[environmental_variables[i]])
    df_owners[environmental_variables[i]] = np.where(df_owners[environmental_variables[i]] == "hoch", "mittel", df_owners[environmental_variables[i]])
for i in range(len(environmental_variables)):
    #ja, gering and ja,bedeutend are labels for oekpd4
    df_companies[environmental_variables[i]].replace(to_replace=["nein", "gering", "ja, gering", "mittel", "ja, bedeutend"], value=[0, 1, 1, 2, 2], inplace=True)
    df_owners[environmental_variables[i]].replace(to_replace=["nein", "gering", "ja, gering", "mittel", "ja, bedeutend"], value=[0, 1, 1, 2, 2], inplace=True)

The flag `b_is_main_owner` is used to seperate minority from majority shareholders (defined as the owner of at least 50% of the equity), and, where there is no information on the percentage owned, only owners with following "characteristics" (dt. Eigenschaft) were considered majority: "Owner" (Inhaber), "Shareholder" (Gesellschafter), "Limited Partner" (Kommanditist), "General Partner" (Komplementär), and "Majority Shareholder" (Hauptaktionär)

In [26]:
df_ownership["b_is_main_owner"] = np.where(df_ownership["b_anteil"] >= 50, True, 
                                           np.where((df_ownership["b_anteil"].isna())
                                           & (df_ownership["b_eigenschaft"].str.contains("Inhaber|Gesellschafter|Kommanditist|Komplementär|Hauptaktionär", regex=True)),
                                             True, False))

Specify the start and end year of the participation, as a step to turn the ownership data into a panel. The entries which have no start nor end date will be assigned all the years where there are observations in the MIP data set (first year 1993, last year 2021). The end date is set to 2023 for all participations which didn't end in the observation period or where there is no information so that 2021 is within the start to end range

In [27]:
df_ownership["b_start_year"] = df_ownership["b_beginn"].astype(str).str[:4]
df_ownership["b_end_year"] = df_ownership["b_ende"].astype(str).str[:4]
df_ownership["b_start_year"] = np.where(df_ownership["b_start_year"] == "0.0", 1993, df_ownership["b_start_year"])
df_ownership["b_start_year"] = np.where(df_ownership["b_start_year"] == "nan", 1993, df_ownership["b_start_year"])
df_ownership["b_end_year"] = np.where(df_ownership["b_end_year"] == "0.0", 2023, df_ownership["b_end_year"])
df_ownership["b_end_year"] = np.where(df_ownership["b_end_year"] == "nan", 2023, df_ownership["b_end_year"])

Parse `b_start_year` and `b_end_year` to integers

In [28]:
df_ownership["b_start_year"] = pd.to_numeric(df_ownership["b_start_year"], downcast="integer")
df_ownership["b_end_year"] = pd.to_numeric(df_ownership["b_end_year"], downcast="integer")

The array `survey_years` contains all the years with sample data for the companies in the MIP panel. Now I will create dummy variables for all sample years, so that we transform the ownership data frame later to a panel

In [29]:
survey_years = np.unique(df_companies["jahr"])
for i in range(len(survey_years)):
    df_ownership[str(survey_years[i])] = np.where((df_ownership["b_start_year"] <= survey_years[i]) 
                                                  & (df_ownership["b_end_year"] > survey_years[i]),True, False)

Drop `b_firma` since it the correct information is encoded in `b_person`. 

In [30]:
df_ownership.drop(labels="b_firma", axis=1, inplace=True)

Drop all the entries that are not a main owner

In [31]:
df_ownership = df_ownership[df_ownership["b_is_main_owner"] == True]
df_ownership.drop(labels=["b_is_main_owner"], axis=1, inplace=True)

In [32]:
num_companyid = len(df_ownership["companyid"].unique())
print("Number of unique companyids:", f"{num_companyid:,}")
num_b_crefo = len(df_ownership["b_crefo"].unique())
print("Number of unique owners (b_crefo):", f"{num_b_crefo:,}")
num_ownerid = len(df_ownership["ownerid"].unique())
print("Number of unique owners (ownerid):", f"{num_ownerid:,}")
df_data_loss.loc[len(df_data_loss.index)] = ["Drop all owners that are not a main owner", num_companyid, num_b_crefo, num_ownerid]

Number of unique companyids: 23,195
Number of unique owners (b_crefo): 59,034
Number of unique owners (ownerid): 8,879


If there are with the same `b_crefo` for the same year, keep the entry with the most recent wave

In [33]:
df_ownership.sort_values(by="welle", axis=0, ascending=False, inplace=True)
df_ownership.drop_duplicates(["b_crefo", "companyid"], keep="first", inplace=True)

Drop all companies where there are more than 5 owners, and then assign a number from 1 to 5 for the pivot table afterwards

In [34]:
df_ownership = df_ownership.groupby(["companyid"]).filter(lambda x: len(x) < 6)
df_ownership["owner_number"] = df_ownership.groupby(["companyid"])["b_crefo"].rank(method="min")

In [35]:
num_companyid = len(df_ownership["companyid"].unique())
print("Number of unique companyids:", f"{num_companyid:,}")
num_b_crefo = len(df_ownership["b_crefo"].unique())
print("Number of unique owners (b_crefo):", f"{num_b_crefo:,}")
num_ownerid = len(df_ownership["ownerid"].unique())
print("Number of unique owners (ownerid):", f"{num_ownerid:,}")
df_data_loss.loc[len(df_data_loss.index)] = ["Drop companies with more than five owners", num_companyid, num_b_crefo, num_ownerid]

Number of unique companyids: 22,127
Number of unique owners (b_crefo): 42,741
Number of unique owners (ownerid): 1,538


`df_panel_ownership` now has the structure of a panel data set, after using `df.melt()`

In [36]:
df_panel_ownership = df_ownership.melt(id_vars=['crefo', 'b_crefo', 'b_eigenschaft', 'b_betrag', 'b_anteil', 
                                                'b_beginn','b_ende', 'b_person', 'welle', 'companyid', 
                                                'ownerid', 'b_start_year', 'b_end_year', 'owner_number'],
                                                  var_name="panel_year")

In this step, I filtered the data set to only have the years where there was an observation of the owner (i.e., the owner was owned the company at that year), and drop the `value` variable afterwards. I also removed the owners which do not have a `companyid`.

In [37]:
df_panel_ownership = df_panel_ownership[df_panel_ownership["value"] == True]
df_panel_ownership = df_panel_ownership[df_panel_ownership["companyid"] >= 0]
df_panel_ownership.drop(labels=["value"], axis=1, inplace=True)

In [38]:
num_companyid = len(df_panel_ownership["companyid"].unique())
print("Number of unique companyids:", f"{num_companyid:,}")
num_b_crefo = len(df_panel_ownership["b_crefo"].unique())
print("Number of unique owners (b_crefo):", f"{num_b_crefo:,}")
num_ownerid = len(df_panel_ownership["ownerid"].unique())
print("Number of unique owners (ownerid):", f"{num_ownerid:,}")
df_data_loss.loc[len(df_data_loss.index)] = ["Drop invalid companyids and owners without a corresponding companyid", num_companyid, num_b_crefo, num_ownerid]

Number of unique companyids: 22,074
Number of unique owners (b_crefo): 40,278
Number of unique owners (ownerid): 1,424


Create a pivot table so that with `b_crefo` as value to match afterwards with the ownership table. This enables us to later check for changes in ownership more easily. NaN are filled with zero so that ownership changes can be spotted by subtracting a year's `b_crefo` value by last year's value. 

In [39]:
df_pivot_ownership = df_panel_ownership.pivot(index=["companyid", "panel_year"], columns="owner_number", values="b_crefo").reset_index()
df_pivot_ownership.fillna(0, inplace=True)

Here we add up all the five different `b_crefo` together so that if there is a new owner or an old owner leaves, we can see that through the difference of this sum from one year to the next

In [40]:
df_pivot_ownership["owner_sum"] = df_pivot_ownership[1.0] + df_pivot_ownership[2.0] + df_pivot_ownership[3.0] + df_pivot_ownership[4.0] + df_pivot_ownership[5.0] 
df_pivot_ownership = df_pivot_ownership.assign(diff=df_pivot_ownership.groupby(['companyid']).owner_sum.diff())
df_pivot_ownership.fillna(0, inplace=True)
df_pivot_ownership["is_owner_change"] = np.where(df_pivot_ownership["diff"] != 0, True, False)                           

Left outer merge with companies data: the option `left` when merging the ownership panel data and the companies panel data ensures that all ownership panel entries are kept and match with the available company panel data


In [41]:
df_pivot_ownership["panel_year"] = pd.to_numeric(df_pivot_ownership["panel_year"])
df_merged_companies = pd.merge(df_pivot_ownership, df_companies, how="left", left_on=["panel_year", "companyid"], right_on=["jahr", "companyid"])

Create a ownerid table, where all `b_crefo` values are mapped to NaN or to a `ownerid` value

In [42]:
df_ownerid_table = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPOwn.csv", 
                           encoding="ISO-8859-1")
df_ownerid_table = df_ownerid_table[["b_crefo", "ownerid"]]
df_ownerid_table.drop_duplicates(inplace=True)
df_ownerid_table.fillna(0, inplace=True)

Create columns with `ownerid` entries and rename them after the merger

In [43]:
df_merged_companies.rename({
        1.0 : "b_crefo_1",
        2.0 : "b_crefo_2",
        3.0 : "b_crefo_3",
        4.0 : "b_crefo_4",
        5.0 : "b_crefo_5"
    }, axis=1, inplace=True)
for i in np.arange(start=1, stop=6):
    current_column = "b_crefo_" + str(i)
    df_merged_companies = pd.merge(df_ownerid_table, df_merged_companies, left_on=["b_crefo"], right_on=[current_column], how="right", suffixes=(str(i), ''))

In [44]:
df_merged_companies.drop(["b_crefo5", "b_crefo4", "b_crefo3", "b_crefo2", "b_crefo"], axis=1, inplace=True)
df_merged_companies.rename({
    "ownerid" : "ownerid_1",
    "ownerid2" : "ownerid_2",
    "ownerid3" : "ownerid_3",
    "ownerid4" : "ownerid_4",
    "ownerid5" : "ownerid_5"
}, axis=1, inplace=True)

`is_ownerid_change` indicates if there is a change of ownership of owners which are included in the MIP data set (and therefore have an `ownerid`)

In [45]:
df_merged_companies["ownerid_sum"] = df_merged_companies["ownerid_1"] + df_merged_companies["ownerid_2"] +df_merged_companies["ownerid_3"] + df_merged_companies["ownerid_4"] + df_merged_companies["ownerid_5"]
df_merged_companies = df_merged_companies.assign(ownerid_diff=df_merged_companies.groupby(['companyid']).ownerid_sum.diff())
df_merged_companies["ownerid_diff"] = df_merged_companies["ownerid_diff"].fillna(0)
df_merged_companies["is_ownerid_change"] = np.where(df_merged_companies["ownerid_diff"] != 0, True, False)

Create a table where the `ownerid` are matched with their MIP panel data information 

In [46]:
df_merged_owners = df_merged_companies[["ownerid_1", "ownerid_2", "ownerid_3", "ownerid_4", "ownerid_5", "companyid", "panel_year"]]
df_merged_owners = df_merged_owners.melt(id_vars=["companyid", "panel_year"], value_name="ownerid")
df_merged_owners.drop(df_merged_owners[df_merged_owners["ownerid"] == 0.0].index, inplace=True)
df_merged_owners.drop("variable", axis=1, inplace=True)
df_merged_owners = pd.merge(left=df_merged_owners, right=df_owners, left_on=["panel_year", "ownerid"], right_on=["jahr", "ownerid"], how="left")

Save the end results

In [47]:
# df_merged_companies.to_csv(r"C:\Users\lucas\OneDrive\BA\Data\outputs\merged_companies.csv")
# df_merged_owners.to_csv(r"C:\Users\lucas\OneDrive\BA\Data\outputs\merged_owners.csv")
df_data_loss.to_excel(r"C:\Users\lucas\OneDrive\BA\Data\outputs\data_loss.xlsx")